# Homework 4: Language Modelling in Hangman

Student Name: Jiyu Chen

Student ID: 908066

Python version used: 3.6.3

## General info

<b>Due date</b>: 11pm, Wednesday May 2nd

<b>Submission method</b>: see LMS

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -20% per day

<b>Marks</b>: 5% of mark for class

<b>Overview</b>: In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character level language models, ranging from unigram approaches to higher over n-gram models. Your objective is to create an automatic player which makes the fewest mistakes.

<b>Materials</b>: See the main class LMS page for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages; if your iPython notebook doesn't run on the marker's machine, you will lose marks.  

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Extra credit</b>: Each homework has a task which is optional with respect to getting full marks on the assignment, but that can be used to offset any points lost on this or any other homework assignment (but not the final project or the exam). We recommend you skip over this step on your first pass, and come back if you have time: the amount of effort required to receive full marks (1 point) on an extra credit question will be substantially more than earning the same amount of credit on other parts of the homework.

<b>Updates</b>: Any major changes to the assignment will be announced via LMS. Minor changes and clarifications will be announced in the forum on LMS, we recommend you check the forum regularly.

<b>Academic Misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.


## The Hangman Game

The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game, and a method allowing interactive play. 

In [1]:
# allowing better python 2 & python 3 compatibility 
from __future__ import print_function 

def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of allowed mistakes
        verbose: be chatty vs silent
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

def human(mask, guessed, **kwargs):
    """
    simple function for manual play
    """
    print('Enter your guess:')
    try:
        return raw_input().lower().strip() # python 3
    except NameError:
        return input().lower().strip() # python 2

You can play the game interactively using the following command:

In [12]:
hangman('whatever', human, 8, True)

Starting hangman game. Target is _ _ _ _ _ _ _ _ length 8
You have 8 attempts remaining.
Enter your guess:
f
Guess is f
Sorry, try again.
You have 7 attempts remaining.
Enter your guess:
a
Guess is a
Good guess: _ _ a _ _ _ _ _
You have 7 attempts remaining.
Enter your guess:
w
Guess is w
Good guess: w _ a _ _ _ _ _
You have 7 attempts remaining.
Enter your guess:
h
Guess is h
Good guess: w h a _ _ _ _ _
You have 7 attempts remaining.
Enter your guess:
t
Guess is t
Good guess: w h a t _ _ _ _
You have 7 attempts remaining.
Enter your guess:
e
Guess is e
Good guess: w h a t e _ e _
You have 7 attempts remaining.
Enter your guess:
v
Guess is v
Good guess: w h a t e v e _
You have 7 attempts remaining.
Enter your guess:
r
Guess is r
Good guess: w h a t e v e r
Congratulations, you won.


1

<b>Instructions</b>: We will be using the words occurring in the *Brown* corpus for *training* an artificial intelligence guessing algorithm, and for *evaluating* the quality of the method. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

Your first task is to compute the unique word types occurring in the *Brown* corpus, using `nltk.corpus.Brown`, selecting only words that are entirely comprised of alphabetic characters, and lowercasing the words. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the training set. Your code should print the sizes of the training and test sets.

Feel free to test your own Hangman performance using `hangman(numpy.random.choice(test_set), human, 8, True)`. It is surprisingly difficult (and addictive)!

(0.5 mark)

In [3]:
from nltk.corpus import brown
import re
import numpy as np

def corpusToWords():
    """preprocess the corpus and build a list of lexicons
    return: partitioned lexicons as trainset isalpha
    return: partitioned lexicons as testset
    """
    lexicons = []
    words = brown.words()
    for w in words:
        
        word = w.lower()
        if re.match("^[a-z]+$",word):
            lexicons.append(word)
    lexicons = list(set(lexicons))
    np.random.shuffle(lexicons)
    
    testset = lexicons[0:1000]
    return lexicons, testset
    
trainset, testset = corpusToWords()

print(len(trainset))
print(len(testset))

40234
1000


<b>Instructions</b>: To set a baseline, your first *AI* attempt will be a trivial random method. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `'a'...'z'` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses). You might want to use `numpy.random.choice` for this purpose.

To measure the performance of this (and later) techiques, implement a method that measures the average number of mistakes made by this technique over all the words in the `test_set`. You will want to turn off the printouts for this, using the `verbose=False` option, and increase the cap on the game length to `max_mistakes=26`. Print the average number of mistakes for the random AI, which will become a baseline for the following steps.

(1 mark)

In [4]:
def createAlpha():
    """create a list of 26 English alphabet
    return: alphabet: a list of 26 English word
    """
    alphabet = []
    for letter in range(97,123):
        alphabet.append(chr(letter))
    return alphabet

def ai(mask, guessed, **kwargs):
    """
    simple function for ai play
    """
    alphabet = createAlpha()
    try:
        # randomly choose one alphabet which is
        # not equals to the alphabets that have
        # been chosed before
        guess = np.random.choice(alphabet)
        while guess in guessed:
            guess = np.random.choice(alphabet)
        return guess.strip() # python 3
    except NameError:
        return guess.strip() # python 2:
    

def AIGuess(ai):
    """play hangman via ai and calculate mistakes over all words
    """
    mistakes = []
    for word in testset:
        mistake = 0
        mistake += hangman(word, ai, 26, False)
        mistakes.append(mistake)
        
    mis = evaluate(testset,mistakes)
    print(mis)
        
def evaluate(testset,mistakes):
    """calculate mistakes over all words
    para: testset: a set of lexicons as testset
    para: mistakes: a list of mistakes on different lexicons
    return: mistake_value: average mistake over all words
    """
    return sum(mistakes)/len(testset)
      
AIGuess(ai)

16.694


**Instructions:** As your first real AI, you should train a *unigram* model over the training set.  This requires you to find the frequencies of characters over all training words. Using this model, you should write a guess function that returns the character with the highest probability, after aggregating (summing) the probability of each blank character in the secret word. Print the average number of mistakes the unigram method makes over the test set. Remember to exclude already guessed characters, and use the same evaluation method as above, so the results are comparable. (Hint: it should be much lower than for random).

(1 mark)

In [5]:
from collections import defaultdict
from collections import Counter


def convert_word(w):
    """add special start/end token to each lexicon
    para: w: a lexicon
    return: transformed lexicon
    """
    return ["<s>"] + [alpha.lower() for alpha in w] + ["</s>"]

def get_unigram_count(words):
    """count the existance of different alphabets over a list of words
    para: words: a list of word which is used to perform alphabet count
    return: number of existance of 28 alphabets(including start/end token)
    """
    unigram_counts = Counter()
    unigrams = Counter()

    # collect initial unigram statistics
    for word in words:
        word = convert_word(word)
        for alpha in word:
            unigram_counts[alpha] += 1
        #unigram_counts["</s>"] += 1
    return unigram_counts


def get_unigram_prob(unigram_counts):
    """get unigram probability
    para: unigram counts
    return: unigram count probability and token count
    """
    M = float(sum(unigram_counts.values()))
    for alpha in unigram_counts:
        unigram_counts[alpha] /= M
    return unigram_counts,M
    

unigram_counts = get_unigram_count(trainset)
unigrams,token_counts = get_unigram_prob(unigram_counts)


def gram_pred_list(unigrams):
    """convert probability Counter to probability sorted list
    para: unigrams: probability Counter
    return: sorted unigram probability list
    """
    x = unigrams
    L = sorted(unigrams, key=unigrams.__getitem__, reverse=True)
    try:
        L.remove('<s>')
        L.remove('</s>')
    except:
        pass
    return L

def ai_unigram(mask, guessed, **kwargs):
    """
    simple function for ai play
    """
    
    #convert unigram Counter to sorted list
    predictions = gram_pred_list(unigrams)

    # choose the most probable alphabet
    guess = predictions[len(guessed)]
    return guess.strip()
    

AIGuess(ai_unigram)


10.434


**Instructions:** The length of the secret word is an important clue that we might exploit. Different length words tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. Your job now is to incorporate this idea by conditioning the unigram model on the length of the secret word, i.e., having *different* unigram models for each length of word. You will need to be a little careful at test time, to be robust to the (unlikely) situation that you encounter a word length that you didn't see in training. Create another AI guessing function using this new model, and print its test performance.   

(0.5 marks)

In [6]:

#print(unigram_counts)

def length_count(words):
    """create a dictionary storing the total count of different length of words
    para: words: a list of lexicons
    return length_counts: a dictionary {length: total number of words under same length}
    """
    length_counts = {}
    for word in words:
        # +2 indicate we will also consider start/end token
        length_counts[len(word)+2] = length_counts.get(len(word)+2,0) +1
    return length_counts

# get c(length)
length_counts = length_count(trainset)

def get_counigram_counts(words):
    """count conditioning unigrams
    para: words: trainset
    return: conditioned unigram counts
    """
    unigram_counts = defaultdict(Counter)
    
    # collect bigram counts
    for word in words:
        word = convert_word(word)
        length = len(word)-2
        for alpha in word:
            unigram_counts[length][alpha] += 1           
    return unigram_counts

def get_con_unigrams(unigram_counts):
    """get conditioned unigram probability
    para: unigram_counts
    return: unigram probability
    """
    
    unigram_prob = defaultdict(Counter)
    
    for length in unigram_counts:
        c = sum(unigram_counts[length].values())
        for alpha in unigram_counts[length]:
            unigram_prob[length][alpha] = unigram_counts[length][alpha] / c            
    return unigram_prob

grams = get_counigram_counts(trainset)
condition_unigrams = get_con_unigrams(grams)


def ai_condition_unigram(mask, guessed, **kwargs):
    """
    simple function for ai play
    """
    
    length = len(mask)            
    possible = sorted(condition_unigrams[length], key=condition_unigrams[length].__getitem__, reverse=True)
        
    try:
        # never predict special token
        possible.remove('</s>')
        possible.remove('<s>')
    except:
        pass
    
    #robust unigram
    padding = gram_pred_list(unigrams)
                
    try:
        guess = possible[len(guessed)]
        return guess.strip() # bigram
    except:
        #robust condition
        padding.reverse()
        guess = padding.pop()
        while guess in guessed:
            guess = padding.pop()
        return guess
                    
AIGuess(ai_condition_unigram)


10.35


**Instructions:** Now for the main challenge, using a *ngram* language model over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

Your job is to develop a *ngram* language model over characters, train this over the training words (being careful to handle the start of each word properly, e.g., by padding with sentinel symbols.) You should use linear interpolation to smooth between the higher order and lower order models, and you will have to decide how to weight each component. 

Your guessing AI algorithm should apply your language model to each blank position in the secret word by using as much of the left context as is known. E.g., in `_ e c _ e _ _` we know the full left context for the first blank (context=start of word), we have a context of two characters for the second blank (context=ec), one character for the second last blank (context=e), and no known context for the last one. If we were using a *n=3* order model, we would be able to apply it to the first and second blanks, but would only be able to use the bigram or unigram distributions for the subsequent blanks. As with the unigram model, you should sum over the probability distributions for each blank to find the expected count for each character type, then select the  character with the highest expected count.

Implement the ngram method for *n=3,4* and *5* and evaluate each of these three models over the test set. Do you see any improvement over the unigram methods above?

(2 marks)

In [7]:
def get_bi_counts(words):
    bigram_counts = defaultdict(Counter)
    
    # collect bigram counts
    for word in words:
        word = convert_word(word)
        # generate a list of bigrams
        bigram_list = zip(word[:-1], word[1:])
        # iterate over bigrams
        for bigram in bigram_list:
            first, second = bigram
            bigram_counts[first][second] += 1
            
    return bigram_counts

def bigrams(bigram_counts):
    
    bigram_prob = defaultdict(Counter)
    
    for prev in bigram_counts:
        c = sum(bigram_counts[prev].values())
        for curr in bigram_counts[prev]:
            bigram_prob[prev][curr] = bigram_counts[prev][curr] / c            
    return bigram_prob

bigram_counts = get_bi_counts(trainset)
bigram_prob = bigrams(bigram_counts)

def ai_bigram(mask, guessed, **kwargs):
    """
    simple function for ai play
    """
    
    mask = list(mask)
    
    if mask[0] == '_':
        prev = '<s>'
    else:
        for i in range(len(mask)):
            if mask[i] == '_':
                prev = mask[i-1]
                break
                
    possible = sorted(bigram_prob[prev], key=bigram_prob[prev].__getitem__, reverse=False)
    
    try:
        # never predict special token
        possible.remove('</s>')
    except:
        pass
    
    
    #robust unigram
    padding = gram_pred_list(unigrams)
                
    try:
        guess = possible.pop()
        while guess in guessed:
            guess = possible.pop()
        return guess.strip() # bigram
    except:
        #robust condition
        guess = padding.pop()
        while guess in guessed:
            guess = padding.pop()
        return guess.strip()
     
AIGuess(ai_bigram)





9.576


In [8]:
#ngram ai

from nltk import ngrams
import math

words = trainset


    

def get_ngram(n):
    """get_ngram_counts
    para: n: ngram
    return: counter format ngram counts
    """
    ngram = defaultdict(Counter)
    unigram = Counter()
    for word in words:
        # convert word
        w = ["<s4>","<s3>","<s2>","<s1>"] + [alpha.lower() for alpha in word] + ["</s1>","</s2>","</s3>","</s4>"]
        
        # get ngrams
        template = ngrams(w,n)
        grams = [x for x in template]
        
        # save into Counter
        for gram in grams:
            prev = [x for x in gram]
            #print(prev)
            curr = prev.pop()
            #print(tuple(prev))
            if n > 2:
                ngram[tuple(prev)][curr] += 1
            elif n == 2:
                ngram[prev.pop()][curr] += 1
            elif n == 1:
                unigram[curr] += 1
    if n > 1:
        return ngram
    else:
        return unigram


# get unigrams counts
unigram_counts = get_ngram(1)   
# get bigram counts
bigram_counts = get_ngram(2)
# get tgram_counts
tgram_counts = get_ngram(3)
# get fgram_counts
fgram_counts = get_ngram(4)
# get figram_counts
figram_counts = get_ngram(5)
# get M
token_counts = float(sum(unigrams.values()))








        




    





In [9]:
alphabet = createAlpha()

#token_count = float(sum(unigrams.values()))




def get_log_prob_interp(prev_word, word, unigram_counts, bigram_counts, trigram_counts, fourgram_counts, fivegram_counts, token_count, lambdas):
    """calculate log probability for each grams and return smoothing probability
    para: prev_word: previous alphabets
    para: word: current alphabets
    para: ngram_counts
    para: token_counts: total number of alphabets
    para: lambdas: smoothing weights
    return: smoothed sum of 1-5gram log probability
    """
    fivegram_lambda = lambdas[0]
    fourgram_lambda = lambdas[1]
    trigram_lambda = lambdas[2]
    bigram_lambda = lambdas[3]
    unigram_lambda = lambdas[4]
    #do not need zerogram in this case
    
    
    # start by getting fivegram probability
    sm_fivegram_counts = fivegram_counts[prev_word][word] * fivegram_lambda
    if sm_fivegram_counts == 0.0:
        interp_fivegram_counts = 0
    else:
        interp_fivegram_counts = sm_fivegram_counts / float(fourgram_counts[(prev_word[-4],prev_word[-3],prev_word[-2])][prev_word[-1]])
    
    # fourgram_probability
    sm_fourgram_counts = fourgram_counts[prev_word][word] * fourgram_lambda
    if sm_fourgram_counts == 0.0:
        interp_fourgram_counts = 0
    else:
        interp_fourgram_counts = sm_fourgram_counts / float(trigram_counts[prev_word[-3]][prev_word[-2]][prev_word[-1]])
    
    
    # trigram probability
    sm_trigram_counts = trigram_counts[prev_word][word] * trigram_lambda
    if sm_trigram_counts == 0.0:
        interp_trigram_counts = 0
    else:
        interp_trigram_counts = sm_trigram_counts / float(bigram_counts[prev_word[-2]][prev_word[-1]])
    
    # bigram probability
    sm_bigram_counts = bigram_counts[prev_word[-1]][word] * bigram_lambda
    if sm_bigram_counts == 0.0:
        interp_bigram_counts = 0
    else:
        interp_bigram_counts = sm_bigram_counts / float(unigram_counts[prev_word[-1]])
     
    # unigram probability
    interp_unigram_counts = (unigram_counts[word]/token_counts) * unigram_lambda
    
    
    # sum of 1-5gram log probability
    prob = math.log(interp_fivegram_counts + interp_fourgram_counts + interp_trigram_counts + interp_bigram_counts + interp_unigram_counts)
    return prob
        



def get_word_log_prob_interp(prev, unigram_counts, bigram_counts, trigram_counts, fourgram_counts, fivegram_counts, token_count, lambdas):
    """given previous alphabets, generate a list of sorted probability of each alphabets
    para: prev: previous alphabets
    para: ngram_counts
    para: token_counts: total number of alphabets in the trainset
    para: lambdas: smoothing weight
    return: a list of sorted tuple (probability, alphabet)
    """
    prev_word = prev
    pred = [(get_log_prob_interp(prev_word, 
                                    word, 
                                    unigram_counts, 
                                    bigram_counts,
                                    trigram_counts,
                                    fourgram_counts,
                                    fivegram_counts,
                                    token_count, 
                                    lambdas),word) for word in alphabet]
    
    pred = sorted(pred, key=lambda x: x[0])
    return pred
    



# trigram ai
def trigram_ai(mask, guessed, **kwargs):
    """change lambdas to fit a trigram model
    """
    mask = list(mask)
    mask = ['<s4>','<s3>','<s2>','<s1>']+mask+['</s1>','</s2>','</s3>','</s4>']
    for i in range(len(mask)):
        if mask[i] == '_':
            prev = (mask[i-4],mask[i-3],mask[i-2],mask[i-1])
            break
    possible = get_word_log_prob_interp(prev, 
                                        unigram_counts, 
                                        bigram_counts, 
                                        tgram_counts, 
                                        fgram_counts, 
                                        figram_counts, 
                                        token_counts, 
                                        (0, 0,0.9,0.1,0.000001))   # trigram
    pred = [x for y,x in possible]  
    guess = pred.pop()
    while guess in guessed or guess is "<s4>" or guess is "<s3>" or guess is "<s2>" or guess is "<s1>":
        guess = pred.pop()
    #print(guess)
    return guess
    
AIGuess(trigram_ai)



9.743


In [10]:
#fourgram ai
def fourgram_ai(mask, guessed, **kwargs):
    """change lambdas to fit a fourgram model
    """
    mask = list(mask)
    mask = ['<s4>','<s3>','<s2>','<s1>']+mask+['</s1>','</s2>','</s3>','</s4>']
    for i in range(len(mask)):
        if mask[i] == '_':
            prev = (mask[i-4],mask[i-3],mask[i-2],mask[i-1])
            break
    possible = get_word_log_prob_interp(prev, 
                                        unigram_counts, 
                                        bigram_counts, 
                                        tgram_counts, 
                                        fgram_counts, 
                                        figram_counts, 
                                        token_counts, 
                                        (0, 0.9,0.1,0.01,0.00000000001))   # fourgram
    pred = [x for y,x in possible]  
    guess = pred.pop()
    while guess in guessed or guess is "<s4>" or guess is "<s3>" or guess is "<s2>" or guess is "<s1>":
        guess = pred.pop()
    #print(guess)
    return guess
    
AIGuess(fourgram_ai)

9.575


In [11]:
#fivegram ai
#fourgram ai
def fivegram_ai(mask, guessed, **kwargs):
    """change lambdas to fit a fivegram model
    """
    mask = list(mask)
    mask = ['<s4>','<s3>','<s2>','<s1>']+mask+['</s1>','</s2>','</s3>','</s4>']
    for i in range(len(mask)):
        if mask[i] == '_':
            prev = (mask[i-4],mask[i-3],mask[i-2],mask[i-1])
            break
    possible = get_word_log_prob_interp(prev, 
                                        unigram_counts, 
                                        bigram_counts, 
                                        tgram_counts, 
                                        fgram_counts, 
                                        figram_counts, 
                                        token_counts, 
                                        (0.9, 0.1,0.01,0.001,0.000001))   # fivegram
    pred = [x for y,x in possible]  
    guess = pred.pop()
    while guess in guessed or guess is "<s4>" or guess is "<s3>" or guess is "<s2>" or guess is "<s1>":
        guess = pred.pop()
    #print(guess)
    return guess
    
AIGuess(fivegram_ai)

7.483
